# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some tests with real teams.

## Imports

In [1]:
from os import makedirs
import json
import random
import pandas as pd
from typing import Callable, Tuple, List
from IPython.display import display, Markdown

from model import FootballSchedulerModel, SymetricScheme
from parse import parse_sol, to_df, to_df_mapped

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "ECU", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))
n = len(countries)

In [3]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
	countries_copy = countries.copy()
	random.shuffle(countries_copy)
	return lambda c: countries_copy.index(c), lambda i: countries_copy[i]

def write_raffle(base_path: str, country_of: Callable[[int], str]):
	with open(f"{base_path}/country_of.json", "w") as f:
		json.dump(dict([(i, country_of(i)) for i in range(n)]), f)

def retrieve_raffle(base_path: str) -> Tuple[Callable[[str], int], Callable[[int], str]]:
    with open(f"{base_path}/country_of.json", "r") as f:
        data = json.load(f)
        shuffled_countries = list(data.values())
        return lambda c: shuffled_countries.index(c), lambda i: shuffled_countries[i]

# Represents the random asignment of country <-> id.
index_of, country_of = raffle(countries)

In [4]:
top_teams = [index_of(c) for c in top_countries]

In [5]:
def write_model_and_raffle(
	base_path: str, model: FootballSchedulerModel
):
	makedirs(base_path, exist_ok=True)
	write_raffle(base_path, country_of=country_of)
	model.write_problem(f"{base_path}/model.lp")


def assert_optimized(model: FootballSchedulerModel):
	model.optimize()
	assert model.get_obj_value() == 0

def write_solution(base_path: str, model: FootballSchedulerModel):
    sol_path = f"{base_path}/solution.sol"
    model.write_sol(sol_path)


def recover_solution_as_df(base_path: str, n: int) -> pd.DataFrame:
    solution_path = f"{base_path}/solution.sol"

    sol = parse_sol(solution_path)

    return to_df(
		n = n,
        sol=sol
    )


def recover_solution_as_mapped_df(base_path: str) -> pd.DataFrame:
	solution_path = f"{base_path}/solution.sol"
	country_of_path = f"{base_path}/country_of.json"

	sol = parse_sol(solution_path)
	country_of = json.load(open(country_of_path))

	return to_df_mapped(
		sol=sol,
		country_of=country_of,
	)

def write_solution_to_latex(base_path: str, sol_df: pd.DataFrame):
	with open(f"{base_path}/solution_table.tex", "w+") as f:
		f.write(sol_df.to_latex(index=False))

## French Scheme

In [6]:
base_path_french = "output/french"

In [7]:
french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH, top_teams)
write_model_and_raffle(base_path_french, french_model)
assert_optimized(french_model)
print(
	f"Optimization of the french model with top teams took: {french_model.get_solving_time()} seconds"
)
write_solution(base_path_french, french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french/model.lp
Optimization of the french model with top teams took: 3.263102 seconds


In [8]:
french_df = recover_solution_as_mapped_df(base_path_french)
write_solution_to_latex(base_path_french, french_df)
display(Markdown(french_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PAR    | @BOL | URU  | ARG  | @ECU | VEN  | @PER | CHI  | @BRA | COL  | @URU | @ARG | ECU  | @VEN | PER  | @CHI | BRA  | @COL | BOL  |
| COL    | BRA  | @ECU | @PER | BOL  | ARG  | @CHI | @VEN | URU  | @PAR | ECU  | PER  | @BOL | @ARG | CHI  | VEN  | @URU | PAR  | @BRA |
| ECU    | @ARG | COL  | @URU | PAR  | CHI  | @VEN | BRA  | @PER | BOL  | @COL | URU  | @PAR | @CHI | VEN  | @BRA | PER  | @BOL | ARG  |
| VEN    | @PER | BOL  | @BRA | URU  | @PAR | ECU  | COL  | @CHI | ARG  | @BOL | BRA  | @URU | PAR  | @ECU | @COL | CHI  | @ARG | PER  |
| BRA    | @COL | PER  | VEN  | @CHI | BOL  | @ARG | @ECU | PAR  | URU  | @PER | @VEN | CHI  | @BOL | ARG  | ECU  | @PAR | @URU | COL  |
| CHI    | @URU | ARG  | @BOL | BRA  | @ECU | COL  | @PAR | VEN  | PER  | @ARG | BOL  | @BRA | ECU  | @COL | PAR  | @VEN | @PER | URU  |
| BOL    | PAR  | @VEN | CHI  | @COL | @BRA | URU  | PER  | @ARG | @ECU | VEN  | @CHI | COL  | BRA  | @URU | @PER | ARG  | ECU  | @PAR |
| ARG    | ECU  | @CHI | @PAR | PER  | @COL | BRA  | @URU | BOL  | @VEN | CHI  | PAR  | @PER | COL  | @BRA | URU  | @BOL | VEN  | @ECU |
| PER    | VEN  | @BRA | COL  | @ARG | @URU | PAR  | @BOL | ECU  | @CHI | BRA  | @COL | ARG  | URU  | @PAR | BOL  | @ECU | CHI  | @VEN |
| URU    | CHI  | @PAR | ECU  | @VEN | PER  | @BOL | ARG  | @COL | @BRA | PAR  | @ECU | VEN  | @PER | BOL  | @ARG | COL  | BRA  | @CHI |

In [9]:
base_path_french_basic = "output/french-basic"

In [10]:
basic_french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH)
write_model_and_raffle(base_path_french_basic, basic_french_model)
assert_optimized(basic_french_model)
print(
    f"Optimization of the basic french model took: {basic_french_model.get_solving_time()} seconds"
)
write_solution(base_path_french_basic, basic_french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french-basic/model.lp
Optimization of the basic french model took: 2.911429 seconds


In [11]:
write_solution_to_latex(
    base_path_french_basic, recover_solution_as_df(base_path_french_basic, n)
)

## English Scheme

In [12]:
base_path_english = "output/english"

In [13]:
english_model = FootballSchedulerModel(n, SymetricScheme.ENGLISH, top_teams)
write_model_and_raffle(base_path_english, english_model)

assert_optimized(english_model)
print(
    f"Optimization of the english model with top teams took: {english_model.get_solving_time()} seconds"
)
write_solution(base_path_english, english_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/english/model.lp
Optimization of the english model with top teams took: 3.950883 seconds


In [14]:
english_df = recover_solution_as_mapped_df(base_path_english)
write_solution_to_latex(base_path_english, english_df)
display(Markdown(english_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PAR    | @BRA | VEN  | @ARG | URU  | @COL | ECU  | PER  | @CHI | BOL  | @BOL | BRA  | @VEN | ARG  | @URU | COL  | @ECU | @PER | CHI  |
| COL    | @VEN | PER  | CHI  | @ECU | PAR  | @ARG | @BOL | BRA  | @URU | URU  | VEN  | @PER | @CHI | ECU  | @PAR | ARG  | BOL  | @BRA |
| ECU    | URU  | @BOL | @BRA | COL  | CHI  | @PAR | @ARG | VEN  | PER  | @PER | @URU | BOL  | BRA  | @COL | @CHI | PAR  | ARG  | @VEN |
| VEN    | COL  | @PAR | @BOL | ARG  | @PER | BRA  | URU  | @ECU | @CHI | CHI  | @COL | PAR  | BOL  | @ARG | PER  | @BRA | @URU | ECU  |
| BRA    | PAR  | @URU | ECU  | @PER | BOL  | @VEN | CHI  | @COL | ARG  | @ARG | @PAR | URU  | @ECU | PER  | @BOL | VEN  | @CHI | COL  |
| CHI    | @PER | ARG  | @COL | BOL  | @ECU | URU  | @BRA | PAR  | VEN  | @VEN | PER  | @ARG | COL  | @BOL | ECU  | @URU | BRA  | @PAR |
| BOL    | @ARG | ECU  | VEN  | @CHI | @BRA | PER  | COL  | @URU | @PAR | PAR  | ARG  | @ECU | @VEN | CHI  | BRA  | @PER | @COL | URU  |
| ARG    | BOL  | @CHI | PAR  | @VEN | @URU | COL  | ECU  | @PER | @BRA | BRA  | @BOL | CHI  | @PAR | VEN  | URU  | @COL | @ECU | PER  |
| PER    | CHI  | @COL | @URU | BRA  | VEN  | @BOL | @PAR | ARG  | @ECU | ECU  | @CHI | COL  | URU  | @BRA | @VEN | BOL  | PAR  | @ARG |
| URU    | @ECU | BRA  | PER  | @PAR | ARG  | @CHI | @VEN | BOL  | COL  | @COL | ECU  | @BRA | @PER | PAR  | @ARG | CHI  | VEN  | @BOL |

In [15]:
base_path_english_basic = "output/english-basic"

In [16]:
basic_english_model = FootballSchedulerModel(n, SymetricScheme.ENGLISH)
write_model_and_raffle(base_path_english_basic, basic_english_model)
assert_optimized(basic_english_model)
print(
    f"Optimization of the basic english model took: {basic_english_model.get_solving_time()} seconds"
)
write_solution(base_path_english_basic, basic_english_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/english-basic/model.lp
Optimization of the basic english model took: 3.389762 seconds


In [17]:
write_solution_to_latex(
    base_path_english_basic, recover_solution_as_df(base_path_english_basic, n)
)

## Inverted scheme

In [18]:
base_path_inverted = "output/inverted"

In [19]:
inverted_model = FootballSchedulerModel(n, SymetricScheme.INVERTED, top_teams)
write_model_and_raffle(base_path_inverted, inverted_model)
assert_optimized(inverted_model)
print(
    f"Optimization of the inverted model with top teams took: {inverted_model.get_solving_time()} seconds"
)
write_solution(base_path_inverted, inverted_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/inverted/model.lp
Optimization of the inverted model with top teams took: 35.167536 seconds


In [20]:
inverted_df = recover_solution_as_mapped_df(base_path_inverted)
write_solution_to_latex(base_path_inverted, inverted_df)
display(Markdown(inverted_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PAR    | @PER | ECU  | VEN  | @ARG | @COL | BOL  | BRA  | @URU | CHI  | @CHI | URU  | @BRA | @BOL | COL  | ARG  | @VEN | @ECU | PER  |
| COL    | @BRA | PER  | ECU  | @BOL | PAR  | @VEN | @CHI | ARG  | @URU | URU  | @ARG | CHI  | VEN  | @PAR | BOL  | @ECU | @PER | BRA  |
| ECU    | ARG  | @PAR | @COL | URU  | PER  | @BRA | @VEN | CHI  | @BOL | BOL  | @CHI | VEN  | BRA  | @PER | @URU | COL  | PAR  | @ARG |
| VEN    | URU  | @ARG | @PAR | CHI  | @BOL | COL  | ECU  | @BRA | @PER | PER  | BRA  | @ECU | @COL | BOL  | @CHI | PAR  | ARG  | @URU |
| BRA    | COL  | @CHI | BOL  | @PER | @URU | ECU  | @PAR | VEN  | @ARG | ARG  | @VEN | PAR  | @ECU | URU  | PER  | @BOL | CHI  | @COL |
| CHI    | @BOL | BRA  | PER  | @VEN | @ARG | URU  | COL  | @ECU | @PAR | PAR  | ECU  | @COL | @URU | ARG  | VEN  | @PER | @BRA | BOL  |
| BOL    | CHI  | @URU | @BRA | COL  | VEN  | @PAR | @ARG | PER  | ECU  | @ECU | @PER | ARG  | PAR  | @VEN | @COL | BRA  | URU  | @CHI |
| ARG    | @ECU | VEN  | @URU | PAR  | CHI  | @PER | BOL  | @COL | BRA  | @BRA | COL  | @BOL | PER  | @CHI | @PAR | URU  | @VEN | ECU  |
| PER    | PAR  | @COL | @CHI | BRA  | @ECU | ARG  | URU  | @BOL | VEN  | @VEN | BOL  | @URU | @ARG | ECU  | @BRA | CHI  | COL  | @PAR |
| URU    | @VEN | BOL  | ARG  | @ECU | BRA  | @CHI | @PER | PAR  | COL  | @COL | @PAR | PER  | CHI  | @BRA | ECU  | @ARG | @BOL | VEN  |

In [21]:
base_path_inverted_basic = "output/inverted-basic"

In [22]:
basic_inverted_model = FootballSchedulerModel(n, SymetricScheme.INVERTED, top_teams)
write_model_and_raffle(base_path_inverted_basic, basic_inverted_model)
assert_optimized(basic_inverted_model)
print(
    f"Optimization of the basic inverted model took: {basic_inverted_model.get_solving_time()} seconds"
)
write_solution(base_path_inverted_basic, basic_inverted_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/inverted-basic/model.lp
Optimization of the basic inverted model took: 37.587129 seconds


In [23]:
write_solution_to_latex(
    base_path_inverted_basic, recover_solution_as_df(base_path_inverted_basic, n)
)

## Back to Back scheme

In [24]:
base_path_b2b = "output/b2b"

In [25]:
b2b_model = FootballSchedulerModel(n, SymetricScheme.BACK_TO_BACK, top_teams)
write_model_and_raffle(base_path_b2b, b2b_model)
assert_optimized(b2b_model)
print(
    f"Optimization of the b2b model with top teams took: {b2b_model.get_solving_time()} seconds"
)
write_solution(base_path_b2b, b2b_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/b2b/model.lp
Optimization of the b2b model with top teams took: 0.219665 seconds


In [26]:
b2b_df = recover_solution_as_mapped_df(base_path_b2b)
write_solution_to_latex(base_path_b2b, b2b_df)
display(Markdown(b2b_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PAR    | @PER | PER  | VEN  | @VEN | @COL | COL  | ARG  | @ARG | @URU | URU  | BRA  | @BRA | BOL  | @BOL | @CHI | CHI  | ECU  | @ECU |
| COL    | @ARG | ARG  | @CHI | CHI  | PAR  | @PAR | @VEN | VEN  | PER  | @PER | @ECU | ECU  | URU  | @URU | BOL  | @BOL | BRA  | @BRA |
| ECU    | BOL  | @BOL | URU  | @URU | CHI  | @CHI | @BRA | BRA  | @VEN | VEN  | COL  | @COL | @PER | PER  | @ARG | ARG  | @PAR | PAR  |
| VEN    | @CHI | CHI  | @PAR | PAR  | ARG  | @ARG | COL  | @COL | ECU  | @ECU | BOL  | @BOL | BRA  | @BRA | @URU | URU  | @PER | PER  |
| BRA    | @URU | URU  | ARG  | @ARG | @BOL | BOL  | ECU  | @ECU | CHI  | @CHI | @PAR | PAR  | @VEN | VEN  | PER  | @PER | @COL | COL  |
| CHI    | VEN  | @VEN | COL  | @COL | @ECU | ECU  | PER  | @PER | @BRA | BRA  | URU  | @URU | @ARG | ARG  | PAR  | @PAR | @BOL | BOL  |
| BOL    | @ECU | ECU  | PER  | @PER | BRA  | @BRA | @URU | URU  | ARG  | @ARG | @VEN | VEN  | @PAR | PAR  | @COL | COL  | CHI  | @CHI |
| ARG    | COL  | @COL | @BRA | BRA  | @VEN | VEN  | @PAR | PAR  | @BOL | BOL  | PER  | @PER | CHI  | @CHI | ECU  | @ECU | @URU | URU  |
| PER    | PAR  | @PAR | @BOL | BOL  | URU  | @URU | @CHI | CHI  | @COL | COL  | @ARG | ARG  | ECU  | @ECU | @BRA | BRA  | VEN  | @VEN |
| URU    | BRA  | @BRA | @ECU | ECU  | @PER | PER  | BOL  | @BOL | PAR  | @PAR | @CHI | CHI  | @COL | COL  | VEN  | @VEN | ARG  | @ARG |

In [27]:
base_path_b2b_basic = "output/b2b-basic"

In [28]:
basic_b2b_model = FootballSchedulerModel(n, SymetricScheme.BACK_TO_BACK, top_teams)
write_model_and_raffle(base_path_b2b_basic, basic_b2b_model)
assert_optimized(basic_b2b_model)
print(
    f"Optimization of the basic b2b model took: {basic_b2b_model.get_solving_time()} seconds"
)
write_solution(base_path_b2b_basic, basic_b2b_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/b2b-basic/model.lp
Optimization of the basic b2b model took: 0.212646 seconds


In [29]:
write_solution_to_latex(
    base_path_b2b_basic, recover_solution_as_df(base_path_b2b_basic, n)
)

## Min-Max Scheme

In [30]:
base_path_min_max = 'output/min-max'

In [31]:
min_max_model = FootballSchedulerModel(n, SymetricScheme.MIN_MAX, top_teams, c=5, d=13)
write_model_and_raffle(base_path_min_max, min_max_model)
assert_optimized(min_max_model)
print(
    f"Optimization of the Min-Max model with top teams took: {min_max_model.get_solving_time()} seconds"
)
write_solution(base_path_min_max, min_max_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/min-max/model.lp
Optimization of the Min-Max model with top teams took: 8816.845848 seconds


In [32]:
min_max_df = recover_solution_as_mapped_df(base_path_min_max)
write_solution_to_latex(base_path_min_max, min_max_df)
display(Markdown(min_max_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| PAR    | BRA  | @COL | VEN  | @ECU | CHI  | @BOL | @URU | PER  | ARG  | @VEN | @CHI | BOL  | @BRA | COL  | @ARG | ECU  | URU  | @PER |
| COL    | @ECU | PAR  | @PER | BOL  | @VEN | BRA  | CHI  | @ARG | @URU | PER  | VEN  | @BRA | ECU  | @PAR | URU  | @BOL | @CHI | ARG  |
| ECU    | COL  | @VEN | @URU | PAR  | @BRA | PER  | ARG  | @CHI | @BOL | URU  | BRA  | @PER | @COL | VEN  | BOL  | @PAR | @ARG | CHI  |
| VEN    | @CHI | ECU  | @PAR | PER  | COL  | @ARG | @BOL | URU  | @BRA | PAR  | @COL | ARG  | CHI  | @ECU | BRA  | @PER | BOL  | @URU |
| BRA    | @PAR | URU  | ARG  | @CHI | ECU  | @COL | @PER | BOL  | VEN  | @ARG | @ECU | COL  | PAR  | @URU | @VEN | CHI  | PER  | @BOL |
| CHI    | VEN  | @ARG | @BOL | BRA  | @PAR | URU  | @COL | ECU  | @PER | BOL  | PAR  | @URU | @VEN | ARG  | PER  | @BRA | COL  | @ECU |
| BOL    | ARG  | @PER | CHI  | @COL | @URU | PAR  | VEN  | @BRA | ECU  | @CHI | URU  | @PAR | @ARG | PER  | @ECU | COL  | @VEN | BRA  |
| ARG    | @BOL | CHI  | @BRA | URU  | @PER | VEN  | @ECU | COL  | @PAR | BRA  | PER  | @VEN | BOL  | @CHI | PAR  | @URU | ECU  | @COL |
| PER    | @URU | BOL  | COL  | @VEN | ARG  | @ECU | BRA  | @PAR | CHI  | @COL | @ARG | ECU  | URU  | @BOL | @CHI | VEN  | @BRA | PAR  |
| URU    | PER  | @BRA | ECU  | @ARG | BOL  | @CHI | PAR  | @VEN | COL  | @ECU | @BOL | CHI  | @PER | BRA  | @COL | ARG  | @PAR | VEN  |

In [33]:
base_path_min_max_basic = "output/min-max-basic"

In [34]:
basic_min_max_model = FootballSchedulerModel(n, SymetricScheme.MIN_MAX, c=5, d=13)
write_model_and_raffle(base_path_min_max_basic, basic_min_max_model)
assert_optimized(basic_min_max_model)
print(
    f"Optimization of the basic Min-Max model took: {basic_min_max_model.get_solving_time()} seconds"
)
write_solution(base_path_min_max_basic, basic_min_max_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/min-max-basic/model.lp
Optimization of the basic Min-Max model took: 10107.567396 seconds


In [35]:
write_solution_to_latex(
    base_path_min_max_basic, recover_solution_as_df(base_path_min_max_basic, n)
)

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] Pyscipopt Docs.